In [123]:
library(ggstatsplot)
library(ggside)
library(dplyr)
library(summarytools)
library(ggfortify)
library(plyr)
library(purrr)
library(hash)
library(scales)
library(ggridges)
library(ggplot2)
library(stringr)

In [124]:
ANALYZE_ORIGINAL_DATA = FALSE
if(ANALYZE_ORIGINAL_DATA){
    psi_dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/"
}else{
    psi_dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi_IRcorrected/"
    
}
setwd(paste0(psi_dir, "/age_tissue_subtype_dependent_exon_usage"))
skeletal_muscle_age_group_assoc.1 <- read.table(paste0(psi_dir, "/skeletal_muscle_age_group_assoc.1.tsv"))
cardiac_muscle_age_group_assoc.1 <- read.table(paste0(psi_dir, "/cardiac_muscle_age_group_assoc.1.tsv"))
skeletal_muscle_age_group_assoc.1 <- skeletal_muscle_age_group_assoc.1 %>% mutate(delta_adult_vs_fetal=mean.adult-mean.fetal_neonatal) %>% mutate(delta_adult_vs_pediatric=mean.adult-mean.pediatric) %>% mutate(delta_pediatric_vs_fetal=mean.pediatric-mean.fetal_neonatal)
cardiac_muscle_age_group_assoc.1 <- cardiac_muscle_age_group_assoc.1 %>% mutate(delta_adult_vs_fetal=mean.adult-mean.fetal_neonatal) %>% mutate(delta_adult_vs_pediatric=mean.adult-mean.pediatric) %>% mutate(delta_pediatric_vs_fetal=mean.pediatric-mean.fetal_neonatal)
skeletal_cardiac_stat <- full_join(skeletal_muscle_age_group_assoc.1, cardiac_muscle_age_group_assoc.1, 
                                   by=c("name"="name"),
                                   suffix = c(".skeletal", ".cardiac")
                                  )


In [125]:
skeletal_cardiac_stat_by_exon <- skeletal_cardiac_stat %>% 
mutate(delta_skeletal_vs_cardiac.fetal_neonatal=mean.fetal_neonatal.skeletal-mean.fetal_neonatal.cardiac) %>% 
mutate(delta_skeletal_vs_cardiac.pediatric=mean.pediatric.skeletal-mean.pediatric.cardiac) %>% 
mutate(delta_skeletal_vs_cardiac.adult=mean.adult.skeletal-mean.adult.cardiac)

colnames(skeletal_cardiac_stat_by_exon)[4:5] <- c("gene", "exon_num")
skeletal_cardiac_stat_by_exon$gene <- as.factor(skeletal_cardiac_stat_by_exon$gene)
stat_by_gene <- skeletal_cardiac_stat_by_exon %>%
  dplyr::group_by(gene) %>%
  dplyr::summarize(
    mean_fetal_neonatal_skeletal = mean(mean.fetal_neonatal.skeletal, na.rm = TRUE),
    mean_pediatric_skeletal = mean(mean.pediatric.skeletal, na.rm = TRUE),
    mean_adult_skeletal = mean(mean.adult.skeletal, na.rm = TRUE),
    delta_adult_vs_fetal.skeletal = mean(delta_adult_vs_fetal.skeletal, na.rm = TRUE),
    delta_adult_vs_pediatric.skeletal = mean(delta_adult_vs_pediatric.skeletal, na.rm = TRUE),
    delta_pediatric_vs_fetal.skeletal = mean(delta_pediatric_vs_fetal.skeletal, na.rm = TRUE),
    mean_fetal_neonatal_cardiac = mean(mean.fetal_neonatal.cardiac, na.rm = TRUE),
    mean_pediatric_cardiac = mean(mean.pediatric.cardiac, na.rm = TRUE),
    mean_adult_cardiac = mean(mean.adult.cardiac, na.rm = TRUE),
    delta_adult_vs_fetal.cardiac = mean(delta_adult_vs_fetal.cardiac, na.rm = TRUE),
    delta_adult_vs_pediatric.cardiac = mean(delta_adult_vs_pediatric.cardiac, na.rm = TRUE),
    delta_pediatric_vs_fetal.cardiac = mean(delta_pediatric_vs_fetal.cardiac, na.rm = TRUE),
    delta_skeletal_vs_cardiac.fetal_neonatal = mean(delta_skeletal_vs_cardiac.fetal_neonatal, na.rm = TRUE),
    delta_skeletal_vs_cardiac.pediatric = mean(delta_skeletal_vs_cardiac.pediatric, na.rm = TRUE),
    delta_skeletal_vs_cardiac.adult = mean(delta_skeletal_vs_cardiac.adult, na.rm = TRUE),    
  )



In [126]:
skeletal_cardiac_stat_by_exon$exon_num_tmp <- as.numeric(skeletal_cardiac_stat_by_exon$exon_num)
skeletal_cardiac_stat_by_exon_ordered <- skeletal_cardiac_stat_by_exon %>%
  arrange(gene, exon_num_tmp)
cols_of_interest <- c('gene', 'exon_num','mean.fetal_neonatal.skeletal', 'mean.pediatric.skeletal', 'mean.adult.skeletal', 'sd.fetal_neonatal.skeletal', 'sd.pediatric.skeletal', 'sd.adult.skeletal', 'delta_adult_vs_fetal.skeletal', 'delta_adult_vs_pediatric.skeletal', 'delta_pediatric_vs_fetal.skeletal', 'mean.fetal_neonatal.cardiac', 'mean.pediatric.cardiac', 'mean.adult.cardiac', 'sd.fetal_neonatal.cardiac', 'sd.pediatric.cardiac', 'sd.adult.cardiac', 'delta_adult_vs_fetal.cardiac', 'delta_adult_vs_pediatric.cardiac', 'delta_pediatric_vs_fetal.cardiac', 'delta_skeletal_vs_cardiac.fetal_neonatal', 'delta_skeletal_vs_cardiac.pediatric', 'delta_skeletal_vs_cardiac.adult')
skeletal_cardiac_stat_by_exon_cleaned <- skeletal_cardiac_stat_by_exon_ordered[, cols_of_interest]


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [127]:
Sys.setenv(psi_fig_tables_RData_dir="/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/fig_tables/RData")
save(skeletal_cardiac_stat_by_exon_cleaned, stat_by_gene, file=paste0(Sys.getenv("psi_fig_tables_RData_dir"), "/ori_data.", ANALYZE_ORIGINAL_DATA, ".age_tissue_subtype_dep.RData"))



In [128]:
skeletal_cardiac_stat_by_exon_cleaned$exon_num <- sub("^0+", "", skeletal_cardiac_stat_by_exon_cleaned$exon_num)
write.table(skeletal_cardiac_stat_by_exon_cleaned, 
            file=paste0(Sys.getenv("psi_fig_tables_RData_dir"), "/ori_data.", ANALYZE_ORIGINAL_DATA, ".skeletal_cardiac_stat_by_exon_cleaned.tsv"), 
            sep = "\t", 
            col.names = TRUE, 
            row.names = FALSE, 
            quote = FALSE)

write.table(stat_by_gene, 
            file=paste0(Sys.getenv("psi_fig_tables_RData_dir"), "/ori_data.", ANALYZE_ORIGINAL_DATA, ".stat_by_gene.tsv"), 
            sep = "\t", 
            col.names = TRUE, 
            row.names = FALSE, 
            quote = FALSE)

